### Cargar dataframe ports_dataframe_input.csv

In [1]:
import pandas as pd
import numpy as np

In [2]:
ports_dataframe = pd.read_csv('ports_dataframe_input.csv', sep=',')

In [3]:
ports_dataframe.drop(columns=['Unnamed: 0'], inplace=True)

Adjuntar datos de 'cargo vessels' y 'freight in ports'

In [8]:
us_containers_2018_df = pd.read_excel('../ais_noaa_gov/p16021coll2_4529_containers_2018.xls',
                                    sheet_name ='By Port TEUS',
                                    header=2,
                                    usecols="A:K",
                                    skiprows=[82,83],
                                    engine='xlrd')

In [12]:
us_containers_2019_df = pd.read_excel('../ais_noaa_gov/p16021coll2_6754_containers_2019.xls',
                                    sheet_name ='By Port TEUS',
                                    header=2,
                                    usecols="A:K",
                                    skiprows=[84,85,86,87,88],
                                    engine='xlrd')

In [14]:
us_containers_2020_df = pd.read_excel('../ais_noaa_gov/p16021coll2_7439_containers_2020.xls',
                                    sheet_name ='By Port TEUS',
                                    header=2,
                                    usecols="A:K",
                                    skiprows=[78],
                                    engine='xlrd')

In [19]:
us_containers_2021_df = pd.read_excel('../ais_noaa_gov/p16021coll2_12774_containers_2021.xls',
                                    sheet_name ='2021 TEUs by Port',
                                    header=2,
                                    usecols="A:K",
                                    skiprows=[121,122,123,124],
                                    engine='xlrd')

In [24]:
us_containers_2022_df = pd.read_excel('../ais_noaa_gov/p16021coll2_14565_containers_2022.xls',
                                    sheet_name ='2022 TEUs by Port',
                                    header=2,
                                    usecols="A:K",
                                    engine='xlrd')

In [26]:
us_containers_2018_df['Year'] = 2018
us_containers_2019_df['Year'] = 2019
us_containers_2020_df['Year'] = 2020
us_containers_2021_df['Year'] = 2021
us_containers_2022_df['Year'] = 2022

In [27]:
us_containers_dataframe = pd.concat([us_containers_2018_df, us_containers_2019_df, us_containers_2020_df, us_containers_2021_df, us_containers_2022_df], 
                                    axis = 0, ignore_index=True, join='outer')

In [28]:
us_containers_dataframe

,PORT NAME,STATE,Loaded InBound DOMESTIC,Empty InBound DOMESTIC,Loaded OutBound DOMESTIC,Empty OutBound DOMESTIC,Total DOMESTIC,Loaded InBound FOREIGN,Loaded OutBound FOREIGN,Total OutBound FOREIGN,Total Loaded Grand,Year
0,Los Angeles,CA,11776.50,0.00,130634.00,0.00,142410.50,4897579.07,1587302.85,6484881.92,6627292.42,2018
1,Long Beach,CA,31218.30,58504.95,185522.05,559.75,275805.05,4104052.21,1274929.59,5378981.80,5595722.15,2018
2,New York (NY and NJ),NY,26753.25,2479.80,27348.25,2479.80,59061.10,3755565.38,1472824.57,5228389.95,5282491.45,2018
3,Savannah,GA,0.00,0.00,0.00,0.00,0.00,2027371.08,1359487.02,3386858.10,3386858.10,2018
4,Houston,TX,0.00,0.00,0.00,0.00,0.00,1209367.71,1042277.63,2251645.34,2251645.34,2018
...,...,...,...,...,...,...,...,...,...,...,...,...
458,"Port Milwaukee, WI",WI,0.00,0.00,0.00,0.00,0.00,6.00,0.00,6.00,6.00,2022
459,"DeTour, MI",MI,0.00,0.00,0.00,0.00,0.00,5.75,0.00,5.75,5.75,2022
460,"Port of Providence, RI",RI,0.00,0.00,0.00,0.00,0.00,3.50,0.00,3.50,3.50,2022
461,"Sault Ste Marie, MI",MI,0.00,0.00,0.00,0.00,0.00,1.75,0.00,1.75,1.75,2022


Una vez cargados los containers de puertos USA, hacer lo mismo con puertos UK:

In [29]:
uk_containers_dataframe = pd.read_csv('../home_uk/uk_port_freight_quaterly.csv', sep=',')

In [34]:
uk_containers_dataframe.dtypes

Major Port            object
2009 Q1              float64
2009 Q2              float64
2009 Q3              float64
2009 Q4              float64
                      ...   
2023 Q4              float64
2024 Q1 [Note 10]    float64
2024 Q2 [Note 10]    float64
2024 Q3 [Note 10]    float64
id_Port              float64
Length: 65, dtype: object

In [38]:
def merge_quarterly_data_sum(df):
    """Merges quarterly data into yearly columns by summing the quarterly values.

    Args:
        df: The input DataFrame with quarterly data columns.

    Returns:
        A new DataFrame with yearly data columns containing the sum of 
        quarterly values, or the original DataFrame if no quarterly columns 
        are found. Returns an error message if the input is not a Pandas DataFrame.
    """

    if not isinstance(df, pd.DataFrame):
        return "Error: Input must be a Pandas DataFrame"

    yearly_data = {}
    for col in df.columns:
        year = col.split(' ')[0]
        if year.isdigit():
            if year not in yearly_data:
                yearly_data[year] = []
            yearly_data[year].append(col)

    new_df = df.copy()

    for year, quarters in yearly_data.items():
        # if len(quarters) == 4:
        # Convert to numeric, coercing errors to NaN. Then sum.
        new_df[year] = new_df[quarters].apply(pd.to_numeric, errors='coerce').sum(axis=1)
        new_df = new_df.drop(quarters, axis=1)
        # else:
        #     print(f"Warning: Not all quarters found for year {year}. Skipping merge.")

    return new_df

In [39]:
merged_df = merge_quarterly_data_sum(uk_containers_dataframe)

In [44]:
uk_containers = merged_df.loc[:,['Major Port','id_Port','2018','2019','2020','2021','2022','2023']]

In [46]:
us_containers_dataframe.columns

Index(['PORT NAME', 'STATE', 'Loaded InBound DOMESTIC',
       'Empty InBound DOMESTIC', 'Loaded OutBound DOMESTIC',
       'Empty OutBound DOMESTIC', 'Total DOMESTIC', 'Loaded InBound FOREIGN',
       'Loaded OutBound  FOREIGN', 'Total OutBound  FOREIGN',
       'Total Loaded Grand', 'Year'],
      dtype='object')

In [48]:
uk_containers_long_dataframe = uk_containers.melt(id_vars=['Major Port','id_Port'], 
                 var_name='Year', 
                 value_name='Total Loaded Grand')

In [53]:
uk_containers_long_dataframe.rename(columns={'Major Port':'PORT NAME'}, inplace=True)

In [54]:
freights_dataframe = pd.concat([us_containers_dataframe, uk_containers_long_dataframe], 
                                    axis = 0, ignore_index=True, join='outer')

In [58]:
freights_dataframe.drop('id_Port', axis=1, inplace=True)

In [59]:
freights_dataframe.to_csv('freights_in_ports.csv',sep=',')

In [113]:
freights_dataframe

,PORT NAME,STATE,Loaded InBound DOMESTIC,Empty InBound DOMESTIC,Loaded OutBound DOMESTIC,Empty OutBound DOMESTIC,Total DOMESTIC,Loaded InBound FOREIGN,Loaded OutBound FOREIGN,Total OutBound FOREIGN,Total Loaded Grand,Year
0,Los Angeles,CA,11776.50,0.00,130634.00,0.00,142410.50,4897579.07,1587302.85,6484881.92,6627292.420,2018
1,Long Beach,CA,31218.30,58504.95,185522.05,559.75,275805.05,4104052.21,1274929.59,5378981.80,5595722.150,2018
2,New York (NY and NJ),NY,26753.25,2479.80,27348.25,2479.80,59061.10,3755565.38,1472824.57,5228389.95,5282491.450,2018
3,Savannah,GA,0.00,0.00,0.00,0.00,0.00,2027371.08,1359487.02,3386858.10,3386858.100,2018
4,Houston,TX,0.00,0.00,0.00,0.00,0.00,1209367.71,1042277.63,2251645.34,2251645.340,2018
...,...,...,...,...,...,...,...,...,...,...,...,...
776,Sunderland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,636.511,2023
777,Swansea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,260.572,2023
778,Tees and Hartlepool,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24021.110,2023
779,Tyne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2599.082,2023


Importar ports_dataframe_input

In [4]:
ports_df_input = pd.read_csv('ports_dataframe_input.csv', sep=',')

In [5]:
ports_df_input.drop(['Unnamed: 0'], axis=1, inplace=True)

In [6]:
ports_df_input.rename({'coord_0':'latitude','coord_1':'longitude'}, inplace=True)

In [115]:
ports_df_input['port_name'] = list_ports

In [86]:
# list_ports = ports_df_input['port_name'].str.split(",", expand=True)[0]
list_ports = list_ports.str.lower()

In [119]:
print(list_ports)

0                  baltimore
1                   beaumont
2                 bellingham
3                     bethel
4             brevig mission
               ...          
220                  wisbech
221               workington
222     rivers hull & humber
223        tees & hartlepool
224    grimsby and immingham
Name: 0, Length: 225, dtype: object


In [ ]:
port_freights = freights_dataframe['PORT NAME'].unique()
port_freights = port_freights.tolist()
port_freights = [item.lower() for item in port_freights]

In [136]:
freights_dataframe

,PORT NAME,STATE,Loaded InBound DOMESTIC,Empty InBound DOMESTIC,Loaded OutBound DOMESTIC,Empty OutBound DOMESTIC,Total DOMESTIC,Loaded InBound FOREIGN,Loaded OutBound FOREIGN,Total OutBound FOREIGN,Total Loaded Grand,Year
0,los angeles,CA,11776.50,0.00,130634.00,0.00,142410.50,4897579.07,1587302.85,6484881.92,6627292.420,2018
1,long beach,CA,31218.30,58504.95,185522.05,559.75,275805.05,4104052.21,1274929.59,5378981.80,5595722.150,2018
2,new york (ny and nj),NY,26753.25,2479.80,27348.25,2479.80,59061.10,3755565.38,1472824.57,5228389.95,5282491.450,2018
3,savannah,GA,0.00,0.00,0.00,0.00,0.00,2027371.08,1359487.02,3386858.10,3386858.100,2018
4,houston,TX,0.00,0.00,0.00,0.00,0.00,1209367.71,1042277.63,2251645.34,2251645.340,2018
...,...,...,...,...,...,...,...,...,...,...,...,...
776,sunderland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,636.511,2023
777,swansea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,260.572,2023
778,tees and hartlepool,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24021.110,2023
779,tyne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2599.082,2023


In [126]:
port_freights.sort()

In [130]:
def normalize_port_name(name):
    parts = name.split(',')
    port_name = parts[0].strip().lower()  # Take the first part (before comma), strip, and lowercase
    return port_name

In [132]:
freights_dataframe['PORT NAME'] = freights_dataframe['PORT NAME'].apply(normalize_port_name)

In [133]:
# Identify numeric columns
numeric_cols = freights_dataframe.select_dtypes(include=['number']).columns

# Group by normalized port name and sum ONLY numeric columns
aggregated_df = freights_dataframe.groupby('PORT NAME')[numeric_cols].sum()

# Handle non-numeric columns (if any) - Example: taking the first value
non_numeric_cols = freights_dataframe.columns.difference(numeric_cols).difference(['PORT NAME']) #exclude 'PORT NAME'
if not non_numeric_cols.empty:  # Check if there are any non-numeric columns
    aggregated_non_numeric = freights_dataframe.groupby('PORT NAME')[non_numeric_cols].first()
    aggregated_df = pd.concat([aggregated_df, aggregated_non_numeric], axis=1) # Concatenate the dataframes

In [135]:
freight_in_port = []
freight_not_in_port = []

In [112]:
for freight in port_freights:
    if freight in list_ports:
        freight_in_port.append(freight)
    else:
        freight_not_in_port.append(freight)
        # print(f"Freight {freight} has no port in list_ports")

In [134]:
aggregated_df

,Loaded InBound DOMESTIC,Empty InBound DOMESTIC,Loaded OutBound DOMESTIC,Empty OutBound DOMESTIC,Total DOMESTIC,Loaded InBound FOREIGN,Loaded OutBound FOREIGN,Total OutBound FOREIGN,Total Loaded Grand,STATE,Year
PORT NAME,,,,,,,,,,,
aberdeen,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,22283.181,None,2018
akutan island,5010.00,9544.00,11848.00,1951.80,28353.80,0.00,0.0,0.00,16858.000,AK,2018
albany port district,0.00,0.00,0.00,0.00,0.00,2.00,0.0,2.00,2.000,NY,2021
anchorage,695353.25,4051.95,214866.70,277787.75,1192059.65,392.01,909.0,1301.01,911520.960,AK,2018
apra harbor,129088.25,21145.00,15827.00,8826.50,174886.75,0.00,0.0,0.00,144915.250,GU,2018
...,...,...,...,...,...,...,...,...,...,...,...
whittier,149165.70,20808.10,23967.35,17774.00,211715.15,2997.80,0.0,2997.80,176130.850,AK,2018
wilmington,0.00,0.00,0.00,0.00,0.00,1538269.16,668301.0,2206570.16,2206570.160,NC,2018
wrangell,11512.45,2021.60,11670.45,7633.60,32838.10,0.00,0.0,0.00,23182.900,AK,2018
